In [1]:
import numpy as np
import pandas as pd
import pickle
import psutil
import random as rn
import tensorflow as tf
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [2]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 
from keras import backend as K

Using TensorFlow backend.


In [3]:
with open('lime_analysis_data/complete_data.pkl', 'rb') as handle:
    complete_data = pickle.load(handle)

In [4]:
with open('shap_analysis_data/results.pkl', 'rb') as handle:
    shap_dict = pickle.load(handle)

In [5]:
x_train = complete_data['xtrain']
x_test = complete_data['xtest']
x_val = complete_data['xval']
y_train = complete_data['ytrain']
y_test = complete_data['ytest']
y_val = complete_data['yval']
# param_dict = complete_data['params_dict']

# Finding good rows - analysis of local right df

In [6]:
from keras.models import load_model
model = load_model('lime_analysis_data/train_v21.h5')

In [7]:
right_actual_df = pd.DataFrame(columns=['right','y_actual'])

In [8]:
right_actual_df['right'] = list(model.predict(x_val).flatten())

In [9]:
right_actual_df['y_actual'] = y_val.reset_index(drop=True)

In [10]:
right_actual_df['right_actual'] = abs((right_actual_df.right - right_actual_df.y_actual)/right_actual_df.y_actual) * 100

In [11]:
right_local_df_trans_subset = right_actual_df[right_actual_df.right_actual<=10]

# Analysis of Intensity df

In [31]:
intensity_df = shap_dict['intensity_df'].copy()

In [32]:
intensity_df_subset = intensity_df.T.loc[list(right_local_df_trans_subset.index),:]

In [33]:
intensity_df_subset_trans = intensity_df_subset.T

In [34]:
intensity_df_subset_trans['sum']=intensity_df_subset_trans.abs().sum(axis=1)

In [35]:
intensity_df_subset_trans

,3,4,5,9,10,14,15,18,21,22,...,270,272,274,275,277,283,284,285,288,sum
LotFrontage,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-12568.841433,...,8053.070084,-74142.863885,-21728.134963,0.000000,10130.091025,0.0,0.0,140713.400072,-32278.943678,2.259200e+06
LotArea,0.000000,-27509.523586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-47354.539745,...,-3666.482512,0.000000,-56028.175902,-53060.817089,0.000000,0.0,0.0,0.000000,0.000000,1.418710e+06
OverallQual,-3829.874309,0.000000,0.000000,31762.696947,-11761.391002,-15529.109586,-3973.113878,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,-13926.305585,0.000000,0.0,0.0,0.000000,0.000000,8.108153e+05
OverallCond,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16026.892650,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,6.022759e+05
YearBuilt,0.000000,0.000000,0.000000,0.000000,-9913.681147,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,4.564876e+05
YearRemodAdd,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,8228.887172,0.0,0.0,0.000000,0.000000,1.546702e+05
MasVnrArea,2181.143422,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,4.569553e+04
BsmtFinSF1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,-10645.149293,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.262009e+05
BsmtFinSF2,0.000000,0.000000,0.000000,0.000000,0.000000,-40838.404786,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,7.879935e+04
BsmtUnfSF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,3.217630e+04


,3,4,5,9,10,14,15,18,21,22,...,270,272,274,275,277,283,284,285,288,sum
FullBath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HalfBath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TotRmsAbvGrd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GarageCars,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OpenPorchSF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EnclosedPorch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MiscVal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MSZoning_C (all),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MSZoning_FV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MSZoning_RL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# List of columns dict

In [12]:
list_of_cols_subset = {}

In [13]:
list_of_cols_subset['drop_none'] = list(x_train.columns)

# Drop non zero

In [39]:
intensity_df_subset_trans_non0 = intensity_df_subset_trans.loc[(intensity_df_subset_trans==0).all(axis=1)]

In [42]:
list_of_cols_subset['drop_non_zero'] = list(intensity_df_subset_trans_non0.index)

# Drop 0 intensities

In [43]:
intensity_df_subset_trans_0 = intensity_df_subset_trans.copy()

In [44]:
intensity_df_subset_trans_0 = intensity_df_subset_trans_0.loc[(intensity_df_subset_trans_0!=0).any(axis=1)]

In [45]:
list_of_cols_subset['drop_0_columns'] = list(intensity_df_subset_trans_0.index)

# Drop last n%

In [25]:
intensity_df_subset_trans_n = intensity_df_subset_trans.copy()

In [ ]:
# for n in range(1,41):
#     list_of_cols_subset['drop_{}_pct_columns'.format(n)] = list(intensity_df_subset_trans_n.iloc[:int(np.ceil(intensity_df_subset_trans_n.shape[0] * (1-(n/100)))),:].index)

In [26]:
for n in range(17,41):
    list_of_cols_subset['drop_{}_pct_columns'.format(n)] = list(intensity_df_subset_trans_n.iloc[:int(np.ceil(intensity_df_subset_trans_n.shape[0] * (1-(n/100)))),:].index)

# Drop None

In [27]:
list_of_cols_subset['drop_none'] = list(intensity_df_subset_trans.index)

In [50]:
list_of_cols_subset.keys()

dict_keys(['drop_none', 'drop_non_zero', 'drop_0_columns'])

# Training with selected columns

In [51]:
# Model Training
def get_search_space():    
    space = {'num_layers': hp.choice('num_layers',['one_hidden', 'two_hidden']),
                'units1': hp.choice('units1', [32, 64, 128, 256,512]),
                'units2': hp.choice('units2', [32, 64, 128, 256,512]),
                'dropout1': hp.uniform('dropout1', .25,.75),
                'dropout2': hp.uniform('dropout2',  .25,.75),
                'batch_size' : hp.choice('batch_size', [16,32,64,128]),
                'nb_epochs' :  500,
                'optimizer': hp.choice('optimizer',['rmsprop', 'adam', 'nadam','sgd']),
                'activation': hp.choice('activation',['relu','sigmoid']),
                'early_stop_rounds': hp.choice('early_stop_rounds',[10,20,30,40,50]),
            }
    return space

In [52]:
def create_model(params):    
#     x_train_temp = x_train.copy()
#     x_test_temp = x_test.copy()
#     y_train_temp = y_train.copy()
#     y_test_temp = y_test.copy()
    model = Sequential()
    model.add(Dense(params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))
    if(params['num_layers'] == 'two_hidden'):
        model.add(Dense(params['units2']))
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dropout2']))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(loss='mse', metrics=['mse'],
                  optimizer=params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=params['early_stop_rounds'])
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=params['batch_size'],
              epochs=500,
              callbacks=[early_stop, history],
              verbose=0,
              validation_data=(x_valid_temp,y_valid_temp)) 
    [loss, mse] = model.evaluate(x_valid_temp,y_valid_temp, verbose=0)
    global num
    mem = psutil.virtual_memory()
    if(np.isnan(mse)):
        print("{}) Validation set root mean sq. error: NaN".format(num),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
        num = num + 1
        return {'loss': np.inf, 'status': STATUS_OK, 'model': model,'params':params}
    print("{}) Validation set root mean sq. error: {:7.2f}".format(num,mse**0.5),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
    num = num + 1
    return {'loss': loss**0.5, 'status': STATUS_OK, 'model': model, 'params':params}

In [53]:
label_col = 'SalePrice'

In [54]:
def get_best_model(label_col):
#     global x_train, x_test, x_valid, y_train, y_test, y_valid
#     input_df, x_train, x_test, x_valid, y_train, y_test, y_valid = data(csv_name=csv_name,label_col=label_col,num_features=num_features)
    trials=Trials()
    space = get_search_space()
    print("Selecting the best network architecture specifically for your data...")
    best = fmin(create_model, space, algo=tpe.suggest, max_evals=50, trials=trials)
    best_model = trials.best_trial['result']['model']
#     scaled_feature_df = pd.concat([x_train,x_test])
#     label_df = pd.concat([y_train,y_test])
#     pred_df = make_predictions(model=best_model,df=scaled_feature_df)
#     output_df = pd.merge(input_df,pred_df['predictions'].to_frame(),left_index=True,right_index=True)
    return best_model, trials

In [56]:
len(list_of_cols_subset['drop_non_zero'])

348

In [ ]:
import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

resultant_dict = {}
num =1 
for n,(key,cols) in enumerate(list_of_cols_subset.items()):
    print(n,key,len(cols))
    global x_train_temp,y_train_temp,x_valid_temp,y_valid_temp, x_test_temp, y_test_temp
    x_train_temp = x_train[cols].copy()
    y_train_temp = y_train.copy()
    x_valid_temp = x_val[cols].copy()
    y_valid_temp = y_val.copy()
    x_test_temp = x_test[cols].copy()
    y_test_temp = y_test.copy()
#     result_model = create_model(x_train_temp,y_train_temp,x_valid_temp,y_valid_temp,param_dict)
    best_model, output_trials = get_best_model(label_col)
    val_rmse = best_model.evaluate(x_valid_temp,y_valid_temp,verbose=0)[0] ** 0.5
    test_rmse = best_model.evaluate(x_test_temp,y_test_temp,verbose=0)[0] ** 0.5
    resultant_dict['{}'.format(key)] = [val_rmse,test_rmse]
    result_lime_analysis_2_param_opt = pd.DataFrame(resultant_dict).T
    result_lime_analysis_2_param_opt.columns = ['valid_rmse','test_rmse']
    result_lime_analysis_2_param_opt.to_csv('shap_analysis_data/result_shap_analysis_2.csv')

0 drop_none 396
Selecting the best network architecture specifically for your data...
1) Validation set root mean sq. error: 23262092427790.68 	Available Mem: 14537.046875 mb
2) Validation set root mean sq. error: 81536.36 	Available Mem: 14524.125 mb
3) Validation set root mean sq. error: 41375.62 	Available Mem: 14514.13671875 mb
4) Validation set root mean sq. error: 203159.12 	Available Mem: 14484.921875 mb
5) Validation set root mean sq. error: NaN 	Available Mem: 14477.203125 mb
6) Validation set root mean sq. error: 201584.10 	Available Mem: 14550.609375 mb
7) Validation set root mean sq. error: 202492.95 	Available Mem: 14517.453125 mb
8) Validation set root mean sq. error: 48977.32 	Available Mem: 14516.0234375 mb
9) Validation set root mean sq. error: 41446.92 	Available Mem: 14500.38671875 mb
10) Validation set root mean sq. error: 202649.34 	Available Mem: 14412.671875 mb
11) Validation set root mean sq. error: 203392.76 	Available Mem: 14402.13671875 mb
12) Validation set 

In [47]:
resultant_dict

{'drop_0_columns': [30404.338886370173, 25088.593289324002],
 'drop_10_pct_columns': [33614.15319895414, 27312.142264047074],
 'drop_11_pct_columns': [33996.761423041055, 27717.11101015293],
 'drop_12_pct_columns': [32965.77000465786, 26775.384902434915],
 'drop_13_pct_columns': [32195.169562721887, 25856.82617934244],
 'drop_14_pct_columns': [33205.293640290714, 31346.555642572934],
 'drop_15_pct_columns': [31352.11639119274, 27208.162252178914],
 'drop_16_pct_columns': [38790.45343629156, 34227.339805454874],
 'drop_1_pct_columns': [32746.81581530301, 26041.753331209744],
 'drop_2_pct_columns': [31989.221972839325, 27428.334694936493],
 'drop_3_pct_columns': [32214.880097503883, 27748.421507627212],
 'drop_4_pct_columns': [30092.675405292164, 24903.079532997603],
 'drop_5_pct_columns': [33934.3148551677, 29022.00700144207],
 'drop_6_pct_columns': [35384.801679583405, 30634.638117514383],
 'drop_7_pct_columns': [30816.73189502182, 26124.21767098418],
 'drop_8_pct_columns': [38453.7855